In [1]:
import re
import json
import urllib2
import sys
from bs4 import BeautifulSoup

In [2]:
def get_info_from_html(html):

    # finds a long digit
    idRE = re.compile(r"1(\d)(\d)(\d)+")
    if idRE.search(html) != None:
        page_index = idRE.search(html).group(0)

    # finds something in between >  <
    # BAD *$&@%$*&% will breka this
    labelRE = re.compile(r">[0-9A-z \'()-/]+<")
    if labelRE.search(html) != None:
        page_label = labelRE.search(html).group(0)
        page_label = page_label[1:-1]
    
    if 'page_index' in locals() and 'page_label' in locals():
        # url = 'http://www.mountainproject.com/v/' + page_index

        # TODO package back should be JSON
        return page_index, page_label #, url
    else:
        return None

In [3]:
def get_children(page_id):
    
    url = 'http://www.mountainproject.com/v/' + str(page_id)
    
    try:
        mp_page = urllib2.urlopen(url)
    except:
        return None
    else:
        mp_html = mp_page.read()
        soup = BeautifulSoup(mp_html, 'html.parser')
        
        website_tree = {}
        
        if url == 'http://www.mountainproject.com/v/':
            dest_iter = [str(s) for s in soup.find_all('span') if re.search("destArea",str(s)) != None]
        elif re.search("leftNavRoutes", mp_html) != None:
            t = str(soup.find(id="leftNavRoutes"))
            dest_iter = t.split("</td><td>")
        elif re.search('leftnavfilternote(.*)',mp_html) != None:
            nav_start = re.search('leftnavfilternote(.*)',mp_html).end()
            nav_end = re.search('(.*)LeftNavDynamicContent',mp_html).start()
            nav_html = mp_html[nav_start:nav_end]
            
            # break out location rows
            TRIM = 26
            dest_iter = nav_html[TRIM:].split('<br/>')
        else:
            return None
        
        for dest in dest_iter:
            features = get_info_from_html(dest)
            
            if features != None:
                page_index, page_label = features
                # print page_label + " " + page_index
                data = { 'page_id':page_index, 'label':page_label }
                website_tree[page_index] = data
            else: # some bullshit
                if re.search('leftnav',dest) == None: 
                    print dest # see if labelRE is missing anything
    
    c = { 'pageID':page_id }
    c['children_info'] = website_tree
    route_list = [c['children_info'][route]['page_id'] for route in c['children_info']]
    c['children'] = route_list

    if page_id in c['children']:
        return None
    else:
        return route_list

In [15]:
def get_info_box_html(mp_html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(mp_html, 'html.parser')
    page_table = soup.find_all('table')
    for t in page_table:
        if re.search("Submitted By:",str(t)) != None:
            return t

In [ ]:
def get_route_info(mp_html):
    
    route_info = {}
    
    info_box_html = get_info_box_html(mp_html)
    
    table_row = info_box_html.find_all('tr')
    for r in table_row:
        morsel = r.get_text()
        morsel = morsel.replace('\n','')
        i = morsel.split(u':\xa0')

        if str(i[0]) != 'Consensus':
            route_info[str(i[0])] = str(i[1])
        else:
            grade = r.get_text()[12:]
            for g in grade.split(u' '):
                h = g.split(u':\xa0')
                if len(h) > 1:
                    route_info['Consensus-'+str(h[0])] = str(h[1])

    return route_info

In [ ]:
def traverse(page_id):
    children = get_children(page_id)
    for child in children:
        if get_children(child) != None:
            traverse(child) # RECURSION!!!
        else:
            #tree = tree.append(get_route_info(page_id))
            return child

tree = traverse(105744222)

Above Tower of Babel 105861997
Across From Cenotaph 105862845
Across From Mind Shaft Cliff Crag 105861965
Across from The Bihedral aka Cascade Crag 105746307
Ament's Rock 105746686
Angel of Death (formerly known as The Beer Can), The 105746077
Animal World 105744917
Arena (aka Contender Wall), The 105745747
Armin's Crag 106056543
Avalon 105745528
Bell Buttress Massif 105864848
Between Crackland and Wizard Rock 106909001
BH Block 106597307
Bihedral Area, The 105859534
Bitty Buttress 105744605
Black Widow Slab 105746409
Blob Rock Area 105810491
Boulder Canyon Boulder 105746474
Boulder Falls East 105959633
Boulder Slips 105860783
Boulderado, The 105744641
Bowling Alley, The 105863546
Brick Wall, The 105744587
Broken Rock 105746331
Bummers Rock 105744576
Canyon Block 106014126
Castle Rock 105744656
Catacombs, The 109142199
Cenotaph Crag 105745813
Cenotaph Spire 106830585
Cob Rock 105744567
Coney Island 105744894
Cornerstone, The 105744776
Couch Fire Crag 105832576
Dome, The 105744581
Eagle

In [12]:
print tree

105746527


In [311]:
import urllib2
import re

page_id = '105732422'
url = 'http://www.mountainproject.com/v/' + page_id
mp_page = urllib2.urlopen(url)
mp_html = mp_page.read()

soup = BeautifulSoup(mp_html, 'html.parser')

if re.search("leftNavRoutes", mp_html):
    t = str(soup.find(id="leftNavRoutes"))
    dest_iter = t.split("</td><td>")

    for dest in dest_iter:
        features = get_info_from_html(dest)
        if features != None:
            print features


('107149327', 'American Flesh Dance (Combination of The Flesh plus American Ghost Dance)')
('107455347', 'Ancient Futures')
('105957640', 'Cutting Edge')
('105732275', 'Dream of Wild Turkeys')
('105732422', 'Epinephrine')
('105732248', 'Fiddler on the Roof')
('105732347', 'Gobbler, The')
('107959290', "I'm Not Worthy")
('105732806', 'Johnny Come Lately')
('105732662', 'Lone Star')
('107443433', 'My Little Pony')
('105732587', 'Overhanging Hangover')
('105732458', 'Prince of Darkness')
('105732389', 'Refried Brains')
('105883684', 'Rock Warrior')
('108382264', 'Sandstone Samurai')
('110029974', 'Shuffle, The')
('108672691', 'Smooth as Silk')
('105732407', 'Sour Mash')
('106145798', 'Spark Plug')
('108501405', 'Steel Monkey')
('107149299', 'Texanephrine')
('106153152', "Texas Hold 'Em")
('106296797', 'Texas Tower Direct')
('105937326', 'Velveeta')
('106714924', 'Velvet Tongue, The')
('105732716', 'Yellow Brick Road')


In [283]:
def get_info_box_html(mp_html):
    from bs4 import BeautifulSoup
    soup = BeautifulSoup(mp_html, 'html.parser')
    page_table = soup.find_all('table')
    for t in page_table:
        if re.search("Submitted By:",str(t)) != None:
            return t

info_box_html = get_info_box(mp_html)
print info_box_html

<table cellpadding="0" cellspacing="0">
<tr>
<td>Type: </td>
<td>Boulder</td>
</tr>
<tr><td>Consensus: </td><td><span class="rateHueco"> <a class="small normal" href="#" onclick="showModalDlgAjax('/scripts/XmlHttpRequest.php?action=ratingOptions', 360, 'Rating Options');return false;">Hueco:</a> V5</span> <span class="rateFont"> <a class="small normal" href="#" onclick="showModalDlgAjax('/scripts/XmlHttpRequest.php?action=ratingOptions', 360, 'Rating Options');return false;">Font:</a> 6C</span> <a class="small" href="/scripts/ShowObjectStats.php?id=107752157">[details]</a></td></tr>
<tr><td valign="top">FA: </td><td><!--MPTEXT--><!--MPTEXTEND--></td></tr>
<tr><td>Page Views: </td><td>511</td></tr>
<tr><td nowrap="nowrap">Submitted By: </td><td><a href="/u/morgan-run-guide//107737661">Morgan Run Guide</a> on Aug 16, 2012</td></tr>
</table>


In [288]:
def get_route_info(mp_html):
    
    route_info = {}
    
    info_box_html = get_info_box_html(mp_html)
    
    table_row = info_box_html.find_all('tr')
    for r in table_row:
        morsel = r.get_text()
        morsel = morsel.replace('\n','')
        i = morsel.split(u':\xa0')

        if str(i[0]) != 'Consensus':
            route_info[str(i[0])] = str(i[1])
        else:
            grade = r.get_text()[12:]
            for g in grade.split(u' '):
                h = g.split(u':\xa0')
                if len(h) > 1:
                    route_info['Consensus-'+str(h[0])] = str(h[1])

    return route_info

get_route_info(mp_html)

{'Consensus-Font': '6C',
 'Consensus-Hueco': 'V5',
 'FA': '',
 'Page Views': '511',
 'Submitted By': 'Morgan Run Guide on Aug 16, 2012',
 'Type': 'Boulder'}

In [331]:
url = 'https://www.mountainproject.com/v/lone-star/105732662'
mp_page = urllib2.urlopen(url)
mp_html = mp_page.read()

soup = BeautifulSoup(mp_html, 'html.parser')

soup.find(itemprop="itemreviewed")
a = soup.find(id="starSummaryText").find_all('meta')
a

[<meta content="3.1" itemprop="average"/>,
 <meta content="4" itemprop="best"/>,
 <meta content="14" itemprop="votes"/>]

In [ ]:
len(tree)